In [ ]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
!python --version

Python 3.10.12


In [ ]:
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/data/'

train_images = np.load(dataset_path + 'train_images.npy')
train_labels = np.load(dataset_path + 'train_labels.npy')
test_images = np.load(dataset_path + 'test_images.npy')
test_labels = np.load(dataset_path + 'test_labels.npy')

Mounted at /content/drive


In [ ]:
print("Train Images Shape:", train_images.shape)
print("Train Labels Shape:", train_labels.shape)
print("Test Images Shape:", test_images.shape)
print("Test Labels Shape:", test_labels.shape)

print("\nTrain Images Data Type:", train_images.dtype)
print("Train Labels Data Type:", train_labels.dtype)
print("Test Images Data Type:", test_images.dtype)
print("Test Labels Data Type:", test_labels.dtype)

print("\nTrain Images Min Value:", np.min(train_images))
print("Train Images Max Value:", np.max(train_images))
print("Test Images Min Value:", np.min(test_images))
print("Test Images Max Value:", np.max(test_images))

print("\nUnique Train Labels:", np.unique(train_labels))
print("Unique Test Labels:", np.unique(test_labels))

Train Images Shape: (28754, 48, 48)
Train Labels Shape: (28754, 7)
Test Images Shape: (7178, 48, 48)
Test Labels Shape: (7178, 7)

Train Images Data Type: float64
Train Labels Data Type: float32
Test Images Data Type: float64
Test Labels Data Type: float32

Train Images Min Value: 0.0
Train Images Max Value: 1.0
Test Images Min Value: 0.0
Test Images Max Value: 1.0

Unique Train Labels: [0. 1.]
Unique Test Labels: [0. 1.]


In [ ]:
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

num_models = 5
models = []
epochs = 40
batch_size = 32

#Data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

#Shuffle the training data
train_images, train_labels = shuffle(train_images, train_labels, random_state=42)

for i in range(num_models):
    print(f"Training model {i+1}/{num_models}")

    #Split the training data into bags
    train_images_bag, _, train_labels_bag, _ = train_test_split(train_images, train_labels, test_size=0.2, random_state=i)

    #Create a new model instance for each bag
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        GlobalAveragePooling2D(),

        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(7, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    train_generator = datagen.flow(
        train_images_bag,
        train_labels_bag,
        batch_size=batch_size
    )

    model.fit(
        train_generator,
        steps_per_epoch=len(train_images_bag) // batch_size,
        epochs=epochs,
        validation_data=(test_images, test_labels)
    )

    models.append(model)

Training model 1/5
Epoch 1/40
718/718 [==============================] - 24s 26ms/step - loss: 2.0089 - accuracy: 0.2263 - val_loss: 1.9778 - val_accuracy: 0.2127
Epoch 2/40
718/718 [==============================] - 18s 25ms/step - loss: 1.6785 - accuracy: 0.3451 - val_loss: 1.8956 - val_accuracy: 0.3465
Epoch 3/40
718/718 [==============================] - 18s 25ms/step - loss: 1.4376 - accuracy: 0.4493 - val_loss: 1.3623 - val_accuracy: 0.4826
Epoch 4/40
718/718 [==============================] - 18s 25ms/step - loss: 1.3117 - accuracy: 0.5013 - val_loss: 1.2603 - val_accuracy: 0.5125
Epoch 5/40
718/718 [==============================] - 17s 24ms/step - loss: 1.2511 - accuracy: 0.5236 - val_loss: 1.1778 - val_accuracy: 0.5476
Epoch 6/40
718/718 [==============================] - 18s 25ms/step - loss: 1.2029 - accuracy: 0.5429 - val_loss: 1.4587 - val_accuracy: 0.4609
Epoch 7/40
718/718 [==============================] - 20s 27ms/step - loss: 1.1708 - accuracy: 0.5579 - val_loss: 1.1

In [ ]:
from google.colab import files

for i, model in enumerate(models):
    model.save(f'emotion_detection_model_{i+1}.h5')

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    with open(f'emotion_detection_model_{i+1}.tflite', 'wb') as f:
        f.write(tflite_model)

#Download the models
for i in range(num_models):
    files.download(f'emotion_detection_model_{i+1}.h5')
    files.download(f'emotion_detection_model_{i+1}.tflite')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>